In [4]:
import pandas as pd
a= pd.read_csv('Authors.csv', sep =';')

In [5]:
a.to_csv('Authors.csv')

In [ ]:
['Université Grenoble Alpes','University of Padua','University of Luxembourg','École Polytechnique','Barcelona Supercomputing Center','Auburn University','Università della Svizzera italiana','Università degli Studi di Enna Kore','University of Pisa','University of Parma', 'Fondazione Bruno Kessler' ,'University of Twente', '']

In [6]:
a['Organization'].unique()

array(['University of Milano-Bicocca',
       'Vita-Salute San Raffaele University', 'University of Twente', nan,
       'Université Grenoble Alpes', 'University of Padua',
       'University of Luxembourg', 'École Polytechnique',
       'Barcelona Supercomputing Center', 'University of Bergamo', 'INAF',
       'SYSBIO.IT', 'University of Pisa', 'Auburn University',
       'University of Florence', 'Università della Svizzera italiana',
       'Università degli Studi di Enna Kore', 'University of Parma',
       'ANSYS', 'Human Technopole', 'Fondazione Bruno Kessler',
       'University of Pavia',
       'National and Kapodistrian University of Athens',
       'Interuniversity Research Centre for Public Services',
       'IRCCS Humanitas Research Hospital',
       'Istituto Superiore di Sanità', 'Instituto Politécnico Nacional',
       'Azienda Socio Sanitaria Territoriale di Monza',
       'University College London', 'Humanitas University', 'Cell Press',
       'Studio dentistico Seraf

In [21]:
import pandas as pd
import requests
import google.generativeai as genai
import os
from dotenv import load_dotenv
load_dotenv()

class UniversityInfoUpdater:
    def __init__(self, dataset, llm_token):
        self.dataset = dataset
        self.llm_token = llm_token
        self.corrected_names_dict = {}
        genai.configure(api_key=llm_token)
        self.model = genai.GenerativeModel('gemini-1.5-flash-latest')

    def find_institution_on_openalex(self, institution_name):
        base_url = "https://api.openalex.org/"
        endpoint = "institutions"
        params = {"filter": f"display_name.search:{institution_name}"}
        response = requests.get(f"{base_url}{endpoint}", params=params)
        if response.status_code == 200:
            data = response.json()
            if "results" in data and len(data["results"]) > 0:
                return data["results"][0]  
        return None

    def extract_correct_university_name(self, text):
        directions_prompt = f"""
        rewrite only the correct name of the university in a correct form in english 
        Text: {text} """
        json_text = self.model.generate_content(directions_prompt).text
        return json_text

    def update_dataset(self):
        
        for index, row in self.dataset.iterrows():
            institution_name = row['Organization']
            print(f"Processing: {institution_name}")
            
           
            institution_info = self.find_institution_on_openalex(institution_name)
            
            if institution_info:
                
                self.dataset.at[index, 'openalex_id'] = institution_info['id']
                self.dataset.at[index, 'country'] = institution_info.get('country_code', 'N/A')
                self.dataset.at[index, 'type'] = institution_info.get('type', 'N/A')
                self.dataset.at[index, 'homepage_url'] = institution_info.get('homepage_url', 'N/A')
                self.dataset.at[index, 'works_count'] = institution_info.get('works_count', 'N/A')
                self.dataset.at[index, 'cited_by_count'] = institution_info.get('cited_by_count', 'N/A')
            else:
                correct_name = self.extract_correct_university_name(institution_name)
                self.corrected_names_dict[institution_name] = correct_name
                print(f"Corrected '{institution_name}' to '{correct_name}'")
                
                
                institution_info = self.find_institution_on_openalex(correct_name)
                if institution_info:
                    self.dataset.at[index, 'openalex_id'] = institution_info['id']
                    self.dataset.at[index, 'country'] = institution_info.get('country_code', 'N/A')
                    self.dataset.at[index, 'type'] = institution_info.get('type', 'N/A')
                    self.dataset.at[index, 'homepage_url'] = institution_info.get('homepage_url', 'N/A')
                    self.dataset.at[index, 'works_count'] = institution_info.get('works_count', 'N/A')
                    self.dataset.at[index, 'cited_by_count'] = institution_info.get('cited_by_count', 'N/A')

    def save_corrected_names(self, filename):
        with open(filename, 'w') as file:
            for wrong_name, correct_name in self.corrected_names_dict.items():
                file.write(f"{wrong_name} -> {correct_name}\n")

    def get_updated_dataset(self):
        return self.dataset

# Example usage
authors = pd.read_csv('Authors.csv')
istitutions_df = pd.DataFrame(authors['Organization'].unique(), columns=['Organization'])
dataset = istitutions_df.loc[0:10]

llm_token = api_key=os.getenv("GEMINI_KEY")
updater = UniversityInfoUpdater(dataset, llm_token)
updater.update_dataset()
updated_dataset = updater.get_updated_dataset()
updated_dataset

Processing: University of Milano-Bicocca


/var/folders/7y/h7b4hxf568n1y6xlpl8vf6340000gn/T/ipykernel_47321/4256210288.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.dataset.at[index, 'openalex_id'] = institution_info['id']
/var/folders/7y/h7b4hxf568n1y6xlpl8vf6340000gn/T/ipykernel_47321/4256210288.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.dataset.at[index, 'country'] = institution_info.get('country_code', 'N/A')
/var/folders/7y/h7b4hxf568n1y6xlpl8vf6340000gn/T/ipykernel_47321/4256210288.py:47: SettingWithCopyWarning: 
A v

Processing: Vita-Salute San Raffaele University
Processing: University of Twente
Processing: nan
Processing: Université Grenoble Alpes
Processing: University of Padua
Processing: University of Luxembourg
Processing: École Polytechnique
Processing: Barcelona Supercomputing Center
Processing: University of Bergamo
Processing: INAF


DefaultCredentialsError: 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.

In [18]:
authors = pd.read_csv('Authors.csv')
istitutions_df = pd.DataFrame(authors['Organization'].unique(), columns=['Organization'])


In [19]:
istitutions_df

,Organization
0,University of Milano-Bicocca
1,Vita-Salute San Raffaele University
2,University of Twente
3,NaN
4,Université Grenoble Alpes
...,...
203,University of Siena
204,Lahore University of Management Sciences
205,University of Ljubljana
206,University of Vienna
